In [1]:
import sys
sys.path.append("..")

from services.segments_database import do_query, insert_trainingdata_from_chapter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.metrics import f1_score, classification_report, confusion_matrix, accuracy_score, roc_auc_score, recall_score, roc_curve, precision_score
import scipy
import numpy as np
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import export_graphviz
from sklearn.model_selection import StratifiedKFold



## Config

In [2]:
number_of_categories = 40
entries_per_category = 50
entries_trainingdata = 50
current_round = 11
current_category = 'zk'
zk_category_id = 4
relevant_columns = ['header_preprocessed', 'parent_preprocessed', 'grandparent_preprocessed', 'words']
random_state = 0

# Functions

In [3]:
def print_tree(model, feature_names, filename): 
    print(model)
    export_graphviz(model, out_file='{}.dot'.format(filename), feature_names = feature_names, class_names = ["0","1"],
                rounded = True, proportion = False, 
                precision = 2, filled = True)

## Get Trainingdata

In [4]:
query = '''
    SELECT *, 
    IF(length(GROUP_CONCAT(word.word SEPARATOR ' ')) > 0, GROUP_CONCAT(word.word SEPARATOR ' '), '')  as words 
    FROM trainingdata 
    LEFT JOIN word ON trainingdata.id = word.chapter_id
    WHERE (label = 'yes' 
    OR label = 'no')
    and round {0} {1}
    GROUP BY trainingdata.id
'''.format('<', current_round)

In [5]:
res = do_query(query)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
print(df.shape)

(934, 33)


In [6]:
df.head()

,id,text,chapter_idx,chapter_number,header,header_preprocessed,parent_header,parent_preprocessed,grandparent_header,grandparent_preprocessed,...,round,suggested_label,createdAt,updatedAt,id,word,tfidf,score,chapter_id,words
0,1418,\nDienstleistungsauftrag.\n,21,21,Auftragsart\n,auftragsart,,,,,...,1,0,2019-05-06 13:14:39,2019-05-10 11:58:56,23108,dienstleistungsauftrag,1.000000,296.0,1418,dienstleistungsauftrag
1,1702,"\nAngebote sind einfach und vollständig, recht...",36,None,Einreichung Angebot\n,einreichung angebot,,,,,...,2,0,2019-06-02 10:49:09,2019-06-02 12:50:03,27722,anbietende,0.101942,316.0,1702,anbietende anbieterin angabe angebot ausgefüll...
2,2267,"\nFreitag, 19. Januar 2018\n",0,None,\n,,,,,,...,1,0,2019-05-06 13:14:38,2019-05-06 15:33:20,37781,19,0.463409,1020.0,2267,19 2018 freitag januar
3,2369,\n1 Referenz des Anbieters in den letzten 5 Ja...,41,None,TECHNISCHE LEISTUNGSFÄHIGKEIT (Firmenreferenze...,technische leistungsfähigkeit firmenreferenzen,,,,,...,6,0,2019-06-02 12:13:33,2019-06-02 14:14:02,39346,1,0.211744,11969.0,2369,1 5 anbieters art aufgabe drei erfüllt jahr ko...
4,2388,\nDie Zuschlagskriterien werden mit Noten zwis...,49,9.3.5,Bewertung der Zuschlagskriterien\n,bewertung zuschlagskriterium,,,,,...,7,1,2019-06-02 12:27:51,2019-06-02 14:31:01,39740,0,0.248047,4432.0,2388,0 1 3 angabe ausreichenden beitrag bewerten be...


## Ensemble Class
source: https://github.com/msahamed/handle_imabalnce_class/blob/master/imbalance_datasets_machine_learning.ipynb

In [7]:
class Create_ensemble(object):
    def __init__(self, n_splits, base_models):
        self.n_splits = n_splits
        self.base_models = base_models

    def predict(self, X, y):
        X = np.array(X.toarray())
        y = np.array(y)
                
        train_pred = np.zeros((X.shape[0], len(self.base_models)))
        f1_scores = np.zeros((len(self.base_models), self.n_splits))
        recall_scores = np.zeros((len(self.base_models), self.n_splits))
        
        for i, clf in enumerate(self.base_models):
            
            folds = StratifiedKFold(n_splits=5, shuffle=True, random_state = random_state).split(X, y)
            
            for j, (train_idx, valid_idx) in enumerate(folds):
                
                X_train = X[train_idx]
                Y_train = y[train_idx]
                X_valid = X[valid_idx]
                Y_valid = y[valid_idx]
                
                clf.fit(X_train, Y_train)
                
                valid_pred = clf.predict(X_valid)
                recall  = recall_score(Y_valid, valid_pred, average='macro')
                f1 = f1_score(Y_valid, valid_pred, average='macro')
                
                recall_scores[i][j] = recall
                f1_scores[i][j] = f1
                
                train_pred[valid_idx, i] = valid_pred
                
                print( "Model- {} and CV- {} recall: {}, f1_score: {}".format(i, j, recall, f1))
            
        return train_pred, recall_scores, f1_scores

## Create Train Data and make TF-IDF

### create X

In [8]:
X_train = df[relevant_columns]
X_train.tail()

,header_preprocessed,parent_preprocessed,grandparent_preprocessed,words
929,111,,,1 2 a baustelleneinrichtung cd ef haus nr plan
930,prozessanforderung,,,0 0074 0158 abbildung ausgewählt beschreibung ...
931,angebotssprache,angebotsabgabe,administrative vorgabe,angebot beilage deutsch einreichen englisch fr...
932,objekt sporzentrum kerenzerberg,,,09 243 3 armaturen einbauten galerie heizgrupp...
933,montage befestigung heizungsdämmung,,,abzukleben aluminium befestigen d dichtigkeits...


### create y

In [9]:
# get df, not series
y_train = df.loc[:,['label']]
y_train.loc[y_train['label'] == 'yes'] = 1
y_train.loc[y_train['label'] == 'no'] = 0
# convert to series
y_train = y_train['label']
y_train[0:5]

0    0
1    0
2    0
3    0
4    0
Name: label, dtype: int64

### tf-idf

train data

In [10]:
words_vec = TfidfVectorizer()
words_tfidf = words_vec.fit_transform(X_train['words'])

header_vec = TfidfVectorizer()
header_tfidf = header_vec.fit_transform(X_train['header_preprocessed'])

parent_vec = TfidfVectorizer()
parent_tfidf = parent_vec.fit_transform(X_train['parent_preprocessed'])

grandparent_vec = TfidfVectorizer()
grandparent_tfidf = grandparent_vec.fit_transform(X_train['grandparent_preprocessed'])

X_train_union = scipy.sparse.hstack([
    words_tfidf,
    header_tfidf,
    parent_tfidf,
    grandparent_tfidf
])

X_train_features = words_vec.get_feature_names()\
               + header_vec.get_feature_names()\
               + parent_vec.get_feature_names()\
               + grandparent_vec.get_feature_names()


print(len(X_train_features))
print(X_train_union.shape) 

7656
(934, 7656)


In [11]:
print(words_tfidf.shape)
print(header_tfidf.shape)
print(parent_tfidf.shape)
print(grandparent_tfidf.shape)

(934, 5903)
(934, 1148)
(934, 412)
(934, 193)


## RandomForest Classifier

In [12]:
classifier0 = RandomForestClassifier(n_estimators=1000, random_state=random_state, class_weight="balanced", max_depth=None)
classifier1 = RandomForestClassifier(n_estimators=1000, random_state=random_state, class_weight="balanced", max_depth=15)
classifier2 = RandomForestClassifier(n_estimators=1000, random_state=random_state, class_weight="balanced", max_depth=10)
classifier3 = RandomForestClassifier(n_estimators=1000, random_state=random_state, class_weight="balanced", max_depth=5)
classifier4 = RandomForestClassifier(n_estimators=1000, random_state=random_state, class_weight="balanced", max_depth=2)
classifier5 = RandomForestClassifier(n_estimators=1000, random_state=random_state, class_weight="balanced", max_depth=1)

## Cross Validation

In [13]:
# base_models = [classifier0, classifier1, classifier2, classifier3, classifier4, classifier5]
base_models = [classifier2]
n_splits = 5
lgb_stack = Create_ensemble(n_splits = n_splits, base_models = base_models)        

In [14]:
y_train_pred, recall_scores, f1_scores = lgb_stack.predict(X_train_union, y_train)

Model- 0 and CV- 0 recall: 0.7143256464011181, f1_score: 0.7239759213037733
Model- 0 and CV- 1 recall: 0.7683047029005914, f1_score: 0.7868541033434651
Model- 0 and CV- 2 recall: 0.7551393973528584, f1_score: 0.772644376899696
Model- 0 and CV- 3 recall: 0.769804822043628, f1_score: 0.797459165154265
Model- 0 and CV- 4 recall: 0.6940298507462687, f1_score: 0.7060931899641576


## Cross Validation Report

In [15]:
df_cv = pd.DataFrame(y_train_pred)
for column in df_cv:
    print('1. The F-1 score of the model {}\n'.format(f1_score(y_train, df_cv[column], average='macro')))
    print('2. The recall score of the model {}\n'.format(recall_score(y_train, df_cv[column], average='macro')))
    print('3. Classification report \n {} \n'.format(classification_report(y_train, df_cv[column])))
    print('4. Confusion matrix \n {} \n'.format(confusion_matrix(y_train, df_cv[column]))) 
    print('===============')

1. The F-1 score of the model 0.7569688917732486

2. The recall score of the model 0.7403852147353986

3. Classification report 
               precision    recall  f1-score   support

           0       0.84      0.92      0.88       671
           1       0.73      0.56      0.64       263

   micro avg       0.82      0.82      0.82       934
   macro avg       0.79      0.74      0.76       934
weighted avg       0.81      0.82      0.81       934
 

4. Confusion matrix 
 [[616  55]
 [115 148]] 



In [16]:
recall_scores

array([[0.71432565, 0.7683047 , 0.7551394 , 0.76980482, 0.69402985]])

In [17]:
#print_tree(lgb_stack.base_models[0].estimators_[50], X_test_features, 'tree_0')
#print_tree(lgb_stack.base_models[1].estimators_[50], X_test_features, 'tree_1')

### print feature importances

In [18]:
for i, model in enumerate(lgb_stack.base_models): 
    print('model {}'.format(i))
    importances = model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
    indices = np.argsort(importances)[::-1]    
    
    for f in range(20):
        print("%d. feature: %s %d (%f)" % (f + 1, X_train_features[indices[f]], indices[f], importances[indices[f]]))
        
    print('===')

model 0
1. feature: zuschlagskriterium 7027 (0.048777)
2. feature: zk 7003 (0.020426)
3. feature: gewichtung 2607 (0.018157)
4. feature: zuschlagskriterium 5805 (0.015792)
5. feature: angebot 550 (0.015432)
6. feature: punkt 4147 (0.015337)
7. feature: zuschlagskriterium 7456 (0.014869)
8. feature: schlüsselperson 4504 (0.013951)
9. feature: qualität 4178 (0.012623)
10. feature: preis 3983 (0.012380)
11. feature: 100 29 (0.011470)
12. feature: max 3440 (0.010617)
13. feature: z3 6991 (0.010586)
14. feature: 50 230 (0.009709)
15. feature: beilage 1193 (0.009395)
16. feature: 10 28 (0.009328)
17. feature: zk 7453 (0.009300)
18. feature: erfüllen 2069 (0.009178)
19. feature: zk 5730 (0.009023)
20. feature: table 4892 (0.008611)
===


## Select Model

In [19]:
final_model = lgb_stack.base_models[0]

# Evaluate model against 1000-Trainingdata

In [20]:
query = '''
    SELECT *, 
    IF(length(GROUP_CONCAT(word.word SEPARATOR ' ')) > 0, GROUP_CONCAT(word.word SEPARATOR ' '), '')  as words
    FROM trainingdata 
    LEFT JOIN word ON trainingdata.id = word.chapter_id
    WHERE (label = 'yes' 
    OR label = 'no') and round = 1000
    GROUP BY trainingdata.id
'''

In [21]:
res = do_query(query)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.shape

(1000, 33)

In [22]:
df.head()

,id,text,chapter_idx,chapter_number,header,header_preprocessed,parent_header,parent_preprocessed,grandparent_header,grandparent_preprocessed,...,round,suggested_label,createdAt,updatedAt,id,word,tfidf,score,chapter_id,words
0,101,\nWeitere spezifische Grundlagen sind vorhande...,26,5.2,Spezifische Grundlagen\n,spezifisch grundlage,Grundlagen\n,grundlage,,,...,1000,0,2019-05-13 10:22:07,2019-05-13 12:28:07,2074.0,ausarbeitung,0.385710,471.0,101.0,ausarbeitung generalplanerangebotes grundlage ...
1,251,\nFolgende Schritte erfolgen bis zum Zuschlags...,26,7.1,Evaluationsphasen\n,evaluationsphase,Evaluation\n,evaluation,,,...,1000,0,2019-05-13 10:22:08,2019-05-13 12:28:10,4544.0,10,0.136392,2510.0,251.0,10 2 3 4 5 6 7 aktivität angebot ausschreibung...
2,509,\n2.6.1 \nAuf das vorliegende Vergabeverfahren...,36,2.6,Ausschreibungsbedingungen\n,ausschreibungsbedingung,,,,,...,1000,0,2019-05-13 10:22:04,2019-05-13 12:28:17,NaN,None,NaN,NaN,NaN,
3,779,\n3.1.1 \n.110 02 Schweizerische Südostbahn AG...,7,3.1,"Bauherr, Besteller, Eigentümer.\n",bauherr besteller eigentümer,"100\tORGANISATION BAUHERR, LAGE, ZWECKBESTIMMU...",100 organisation bauherr lage zweckbestimmung,,,...,1000,0,2019-05-13 10:22:05,2019-05-13 12:28:19,NaN,None,NaN,NaN,NaN,
4,800,\nDie Gasleitung wird im Bereich der Zentralst...,55,55,Gas\n,gas,,,,,...,1000,0,2019-05-13 10:22:03,2019-05-13 12:28:21,13165.0,110,0.253070,790.0,800.0,110 380 bereich erstellen gasleitung grabenlän...


In [23]:
X = df[relevant_columns]
X.tail()


,header_preprocessed,parent_preprocessed,grandparent_preprocessed,words
995,ts mitte steuerung treppenabgang,,,abdeckung ausführung ausgehen aussen bedienste...
996,angebotsblatt,,,abfallsammelfahrzeug anbieter anbieters aufbau...
997,montage befestigung heizungsdämmung,,,abzukleben aluminium befestigen d dichtigkeits...
998,total heizgruppe bodenheizung garderobe übrig ...,,,fr heizgruppe konvektor lichtschacht she
999,unterlage 4 fach inkl revisionspläne unterlage...,,,1 a anlageteil bauleitung bewegen ca klappen l...


In [24]:
# get df, not series
y_test = df.loc[:,['label']]
y_test.loc[y_test['label'] == 'yes'] = 1
y_test.loc[y_test['label'] == 'no'] = 0
# convert to series
y_test = y_test['label']

In [25]:
words_tfidf = words_vec.transform(X['words'])
header_tfidf = header_vec.transform(X['header_preprocessed'])
parent_tfidf = parent_vec.transform(X['parent_preprocessed'])
grandparent_tfidf = grandparent_vec.transform(X['grandparent_preprocessed'])

X_test_union = scipy.sparse.hstack([
    words_tfidf,
    header_tfidf,
    parent_tfidf,
    grandparent_tfidf
])

print(X_test_union.shape)

(1000, 7656)


In [26]:
y_test_pred = final_model.predict(X_test_union)

In [27]:
print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test,y_test_pred))  
print(accuracy_score(y_test, y_test_pred))  

[[960   9]
 [ 16  15]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       969
           1       0.62      0.48      0.55        31

   micro avg       0.97      0.97      0.97      1000
   macro avg       0.80      0.74      0.77      1000
weighted avg       0.97      0.97      0.97      1000

0.975


In [28]:
# select second class only
y_test_pred_proba = final_model.predict_proba(X_test_union)[:,1]

In [29]:
roc_value = roc_auc_score(y_test, y_test_pred_proba)
roc_value

0.8506275175605047

## Print Classifications

In [ ]:
classifications = pd.concat([df.iloc[:,0], X, y_test, pd.Series(y_test_pred, name="pred"), pd.Series(y_test_pred_proba, name="pred_proba")], axis=1)
classifications.head()

In [ ]:
confusion_type = [];
for index, row in classifications.iterrows():
    if row.label == 0 and row.pred == 0:
        confusion_type.append('TN')
    elif row.label == 1 and row.pred == 1: 
        confusion_type.append('TP')
    elif row.label == 0 and row.pred == 1: 
        confusion_type.append('FP')
    elif row.label == 1 and row.pred == 0: 
        confusion_type.append('FN')
classifications = pd.concat([classifications, pd.Series(confusion_type, name='confusion_type')], axis=1)
classifications.head()

In [ ]:
classifications[classifications.confusion_type == 'FN']['id'].tolist()

In [ ]:
classifications.to_csv('predictions_1000.csv')

# select next trainingdata

In [ ]:
results = []
query = '''
  SELECT *, GROUP_CONCAT(word.word SEPARATOR ' ') as words FROM chapter 
    INNER JOIN word ON chapter.id = word.chapter_id
  WHERE topic_1_id = {0}
  GROUP BY chapter.id
  ORDER BY RAND()
  LIMIT {1};
'''


for i in range(0, number_of_categories):
    print('getting data for category {}'.format(i))
    res = do_query(query.format(i, entries_per_category))
    df = pd.DataFrame(res.fetchall())
    df.columns = res.keys()
    results.append(df)
    
df = pd.concat(results, ignore_index=True)
df.head()

Calculate tfidf based on classifiers from previous model

In [ ]:
next_X = df[relevant_columns]
words_tfidf = words_vec.transform(df['words'])
header_tfidf = header_vec.transform(df['header_preprocessed'])
parent_tfidf = parent_vec.transform(df['parent_preprocessed'])
grandparent_tfidf = grandparent_vec.transform(df['grandparent_preprocessed'])

X_next_union = scipy.sparse.hstack([
    words_tfidf,
    header_tfidf,
    parent_tfidf,
    grandparent_tfidf
])

union_features = words_vec.get_feature_names()\
               + header_vec.get_feature_names()\
               + parent_vec.get_feature_names()\
               + grandparent_vec.get_feature_names()


print(len(union_features))
print(X_next_union.shape) 

In [ ]:
y_next_pred_proba = final_model.predict_proba(X_next_union)[:,1]
security = abs(y_next_pred_proba - 0.5) * 2
security

In [ ]:
df['security'] = security
df = df.sort_values(by='security')
df.head()

## Evaluate

In [ ]:
print('mean: {0}'.format(df['security'].mean()))
print('median: {0}'.format(df['security'].median()))

In [ ]:
grouped_sec = pd.DataFrame(df.groupby('topic_1_id')['security'].mean())
grouped_sec['median'] = df.groupby('topic_1_id')['security'].median()
grouped_sec= grouped_sec.sort_values('median', ascending=False)
grouped_sec

## Insert Trainingdata

In [ ]:
counter = 0
for i, d in df[0:entries_trainingdata].iterrows():
    counter += 1
    print('inserting trainingdata {0} of {1}'.format(counter, len(df[0:entries_trainingdata])))
    try:
        insert_trainingdata_from_chapter(
            d,
            current_category,
            current_round + 1,
            d['topic_1_id'] == zk_category_id
        )
    except: 
        e = sys.exc_info()[0]
        print(e)

# Use model for all chapters

In [30]:
query = '''
    select *, group_concat(word.word separator ' ') as words from chapter 
    inner join word on chapter.id = word.chapter_id
    group by chapter.id
'''

In [31]:
res = do_query(query)
df = pd.DataFrame(res.fetchall())
df.columns = res.keys()
df.head()

,id,text,chapter_idx,chapter_number,header,header_preprocessed,parent_header,parent_preprocessed,grandparent_header,grandparent_preprocessed,...,topic_4_value,topic_5_id,topic_5_value,document_id,id,word,tfidf,score,chapter_id,words
0,1,\nStaatskanzlei\n\nRathaus 8750 Glarus\n\nTele...,0,None,\n,,,,,,...,0.000231,32,0.000011,7878,1,055,0.424899,54.0,1,055 09 11 12 15 60 646 8750 fax glarus mail ra...
1,2,\nAusschreibungsgegenstand\n\nDer Kanton Glaru...,1,1,Ausschreibung; Arbeiten zur Installation und z...,ausschreibung arbeit installation betrieb elek...,,,,,...,0.003203,20,0.003034,7878,15,aktiv,0.102934,168.0,2,aktiv anforderung anforderungsbereich dabei do...
2,3,\nDer Regierungsrat will die elektronische Sti...,2,2,Ziele Projektziele:\n,ziel projektziel,,,,,...,0.001608,20,0.001047,7878,35,2018,0.136370,5171.0,3,2018 2019 ausschreibung bund e elektronisch gl...
3,4,"\nEs wird vorausgesetzt, dass der Anbieter gen...",3,3,Marktpositionierung und Rolle des Anbieters\n,marktpositionierung rolle anbieters,,,,,...,0.000610,37,0.000421,7878,54,anbieter,0.198733,6705.0,4,anbieter bereitstellung betrieb dass e genügen...
4,5,\nDie nachfolgende Grafik zeigt die Abgrenzung...,4,4,Systemabgrenzung\n,systemabgrenzung,,,,,...,0.002028,7,0.001550,7878,66,abgrenzung,0.132285,71.0,5,abgrenzung aktivität bestehen betreiben datens...


In [32]:
X = df[relevant_columns]
X.tail()

,header_preprocessed,parent_preprocessed,grandparent_preprocessed,words
262908,q dokument hlkks,,,13 a aufwand ausfüllen bewirtschaften dokument...
262909,eco bauweise,,,anforderung anlage aufgeführtes auflage b berü...
262910,rohrleitungsanpassung,,,000 001 5 500 betrag chf nachtragssumme nachtr...
262911,objekt sporzentrum kerenzerberg,,,06 249 6 los übriges
262912,übertrag,,,000 01 06 2 500 analog anhand anstelle bearbei...


In [33]:
words_tfidf = words_vec.transform(X['words'])
header_tfidf = header_vec.transform(X['header_preprocessed'])
parent_tfidf = parent_vec.transform(X['parent_preprocessed'])
grandparent_tfidf = grandparent_vec.transform(X['grandparent_preprocessed'])

X_union = scipy.sparse.hstack([
    words_tfidf,
    header_tfidf,
    parent_tfidf,
    grandparent_tfidf
])

print(X_union.shape)

(262913, 7656)


In [36]:
y = final_model.predict(X_union)

In [38]:
df['y'] = y

In [39]:
len(df.loc[df['y'] == 1])

6374

In [40]:
df.loc[df['y'] == 1]

,id,text,chapter_idx,chapter_number,header,header_preprocessed,parent_header,parent_preprocessed,grandparent_header,grandparent_preprocessed,...,topic_5_id,topic_5_value,document_id,id,word,tfidf,score,chapter_id,words,y
13,14,\nFormelle Prüfung\n\nDie Nichteinhaltung von ...,6,7,Evaluation der Angebote\n,evaluation angebot,,,,,...,28,0.007192,6836,293,0,0.267809,4432.0,14,0 5 angabe angebot bewertung eignung erfüllen ...,1
103,119,\nDen 10 zugelassenen Anbietern werden die Sub...,33,None,Phase 2: Angebotsanfrage\n,phase 2 angebotsanfrage,,,,,...,25,0.005490,3528,2470,10,0.246558,2510.0,119,10 anbieter angebot bewerten eingereichten fri...,1
106,122,\nDen 10 zugelassenen Anbietern werden die Sub...,34,None,Phase 2: Angebotsanfrage\n,phase 2 angebotsanfrage,,,,,...,25,0.005490,6074,2526,10,0.246383,2510.0,122,10 anbieter angebot bewerten eingereichten fri...,1
111,128,\nDen 10 zugelassenen Anbietern werden die Sub...,33,None,Phase 2: Angebotsanfrage\n,phase 2 angebotsanfrage,,,,,...,25,0.005490,3499,2624,10,0.246558,2510.0,128,10 anbieter angebot bewerten eingereichten fri...,1
194,223,\nIn dieser Phase wird über die Zulassung zur ...,65,None,"Phase 1, Präqualifikation\n",phase 1 präqualifikation,,,,,...,4,0.003409,3528,4038,00,0.236031,4014.0,223,00 1 10 2 20 3 5 anbieter bauen beurteilen eig...,1
195,224,\nAnhand der Zuschlagskriterien findet eine de...,21,5,Zuschlagskriterien\n,zuschlagskriterium,,,,,...,32,0.007372,9498,4045,000,0.120114,3485.0,224,000 01 02 03 10 100 40 400 50 500 administrati...,1
201,232,\nIn dieser Phase wird über die Zulassung zur ...,66,None,"Phase 1, Präqualifikation\n",phase 1 präqualifikation,,,,,...,4,0.003409,6074,4133,00,0.236127,4014.0,232,00 1 10 2 20 3 5 anbieter bauen beurteilen eig...,1
206,239,\nIn dieser Phase wird über die Zulassung zur ...,65,None,"Phase 1, Präqualifikation\n",phase 1 präqualifikation,,,,,...,4,0.003409,3499,4191,00,0.236847,4014.0,239,00 1 10 2 20 3 5 anbieter bauen beurteilen eig...,1
209,242,\nDas im Anforderungskatalog (Anhang 7) aufgef...,23,5.2,Administration und Erfahrung\n,administration erfahrung,Zuschlagskriterien\n,zuschlagskriterium,,,...,23,0.002540,9498,4244,02,0.170669,1159.0,242,02 03 7 administrativ anforderungskatalog anha...,1
221,256,\nBezüglich Erfüllung der qualitativen Zuschla...,28,7.3,Taxonomie Typen\n,taxonomie typ,Evaluation\n,evaluation,,,...,0,0.008322,9498,4680,0,0.163505,4432.0,256,0 100 2 a angebot anhang b durchschnittsprämie...,1


In [41]:
df.loc[df['y'] == 1].to_csv('found_zk.csv')

## plot ROC curve


In [ ]:
# select second class only
y_test_pred_proba = final_model.predict_proba(X_test_union)[:,1]

y_train_pred = final_model.predict(X_train_union)
y_train_pred_proba = final_model.predict_proba(X_train_union)[:,1]

In [ ]:
roc_value = roc_auc_score(y_test, y_pred_proba)
roc_value

In [ ]:
def evaluate_model(predictions, probs, train_predictions, train_probs):
    """Compare machine learning model to baseline performance.
    Computes statistics and shows ROC curve."""
    
    baseline = {}
        
    baseline['recall'] = recall_score(test_labels, 
                                     [1 for _ in range(len(test_labels))])
    baseline['precision'] = precision_score(test_labels, 
                                      [1 for _ in range(len(test_labels))])
    baseline['roc'] = 0.5
    
    results = {}
    
    results['recall'] = recall_score(test_labels, predictions)
    results['precision'] = precision_score(test_labels, predictions)
    results['roc'] = roc_auc_score(test_labels, probs)
    
    train_results = {}
    train_results['recall'] = recall_score(train_labels, train_predictions)
    train_results['precision'] = precision_score(train_labels, train_predictions)
    train_results['roc'] = roc_auc_score(train_labels, train_probs)
    
    for metric in ['recall', 'precision', 'roc']:
        print(f'{metric.capitalize()} Baseline: {round(baseline[metric], 2)} Test: {round(results[metric], 2)} Train: {round(train_results[metric], 2)}')
    
    # Calculate false positive rates and true positive rates
    base_fpr, base_tpr, _ = roc_curve(test_labels, [1 for _ in range(len(test_labels))])
    model_fpr, model_tpr, _ = roc_curve(test_labels, probs)

    plt.figure(figsize = (8, 6))
    plt.rcParams['font.size'] = 16
    
    # Plot both curves
    plt.plot(base_fpr, base_tpr, 'b', label = 'baseline')
    plt.plot(model_fpr, model_tpr, 'r', label = 'model')
    plt.legend();
    plt.xlabel('False Positive Rate'); 
    plt.ylabel('True Positive Rate'); plt.title('ROC Curves');
    plt.show();

In [ ]:
test_labels = y_test
train_labels = y_train
evaluate_model(y_test_pred, y_test_pred_proba, y_train_pred, y_train_pred_proba)